In [1]:
# This is a Python 3 script

In [100]:
# Format code for fortran code: FORMAT(I4,F10.3,F10.3)
def ConstructInputASURV(fitting_table, line1, line2, outdir, obj=None):
    from astropy.io import ascii
    import numpy as np

    data = ascii.read(fitting_table)
    
    # get line fitting results of a single object if specified
    if obj != None:
        data = data[data['Object'] == obj]
    
    
    # Restrict to only select the 1-D spectra
    data = data[data['Pixel_No.'] == 'c']
    
    data1 = data[data['Line'] == line1]
    data2 = data[data['Line'] == line2]
    
    # make sure the objects have both PACS and SPIRE data
    data1_filter = np.zeros_like(data1['Validity'].data, dtype='bool')
    for i in range(len(data1)):
        if data1['Object'][i] in data2['Object'].data:
            data1_filter[i] = True

    data2_filter = np.zeros_like(data2['Validity'].data, dtype='bool')
    for i in range(len(data2)):
        if data2['Object'][i] in data1['Object'].data:
            data2_filter[i] = True
    
    data1 = data1[data1_filter]
    data2 = data2[data2_filter]
    
    # replace the non-detection to upper limit
    def FindUpperLimit(table):
        upperlimit_flag = np.empty_like(table['Str(W/cm2)'].data, dtype=int)
        linestr = np.empty_like(table['Str(W/cm2)'].data, dtype=float)
        linestr_unc = np.empty_like(table['Str(W/cm2)'].data, dtype=float)
        
        for i in range(len(table)):
            # for line fluxes with SNR < 3 or not able to fit properly
            if (table['Validity'][i].astype('float') == '0') or (table['SNR'][i].astype('float') < 3.0):
                upperlimit_flag[i] = -1
                linestr[i] = table['Noise(W/cm2/um)'][i].astype('float')*table['FWHM(um)'][i].astype('float')*1.064*3
                linestr_unc[i] = -1
            else:
                upperlimit_flag[i] = 0
                linestr[i] = table['Str(W/cm2)'][i].astype('float')
                linestr_unc[i] = table['Sig_str(W/cm2)'][i].astype('float')
            
        # convert the flux unit from W/cm2 to W/m2
        linestr = linestr*1e4
        linestr_unc[linestr_unc != -1] = linestr_unc[linestr_unc != -1]*1e4
        
        return upperlimit_flag, linestr, linestr_unc

    ulf1, linestr1, linestr_unc1 = FindUpperLimit(data1)
    ulf2, linestr2, linestr_unc2 = FindUpperLimit(data2)
    
    # check size
    if len(linestr1) != len(linestr2):
        print('The sizes of two arrays are inconsistent.')
        return None
    
#     For bivariate data:

#     3 : Both Independent and Dependent Variables are Lower Limits
#     2 : Only independent Variable is a Lower Limit
#     1 : Only Dependent Variable is a Lower Limit
#     0 : Detected Point
#     -1 : Only Dependent Variable is an Upper Limit
#     -2 : Only Independent Variable is an Upper Limit
#     -3 : Both Independent and Dependent Variables are Upper Limits

    # Define data1 as the independent variable, and data2 as the dependent variable.
    ulf1 = ulf1*2
    ulf = ulf1+ulf2
    
    # writing out
    foo = open(outdir+'co.dat', 'w')
#     foo.write('# '+line1+' and '+line2+'\n')
#     foo.write('# flux in 10^-18 W/m2 unit. \n')
    for i in range(len(linestr1)):
        foo.write('{:>4d}{:>10.3f}{:>10.3f} \n'.format(ulf[i], linestr1[i]*1e18, linestr2[i]*1e18))
    foo.close()
    
    # if no detection found, throw a flag
    flag = True
    if not 0 in ulf:
        flag = False
    return flag

In [86]:
def RunandParseASURV(code_dir, outdir):
    import os
    import subprocess
    import sys
    
    ON_POSIX = 'posix' in sys.builtin_module_names
    # claer out previous output to prevent seg fault
    if os.path.exists(code_dir+'co.out'):
        os.remove(code_dir+'co.out')

#     p = subprocess.call([code_dir+'asurv_code'])
    p = subprocess.Popen([code_dir+'asurv_code_static'], stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE, bufsize=1, close_fds=ON_POSIX)
#     p.communicate()
    print(p.stdout.readlines()[-1])
    print(p.stderr.readlines())
    
    out = open(code_dir+'co.out', 'r').read().splitlines()

    # [print(line) for line in out]

    # find the result about generalized Kendall's Tau, and Spearman's Rho
    for line in out:
        if 'Z-VALUE' in line:
            tau_i = out.index(line)
        if 'SPEARMANS RHO =' in line:
            spear_rho = out.index(line)

    kendall = (float(out[tau_i].split('=')[-1].strip()), float(out[tau_i+1].split('=')[-1].strip()))
    spearman = (float(out[spear_rho].split('=')[-1].strip()), float(out[spear_rho+1].split(' ')[16]))
    line1, line2 = out[8].split('AND')[0].strip(), out[8].split('AND')[1].strip()
#     print(kendall, spearman)
#     os.rename(code_dir+'co.out', outdir+line1+'_'+line2+'.out')
#     os.remove(code_dir+'co.out')

    return kendall, spearman


In [87]:
fitting_table = '/Volumes/SD-Mac/CDF_archive_v2_lines.txt'
line1 = 'CO30-29'
line2 = 'CO35-34'
code_dir = '/Users/yaolun/astro_tools/'
outdir = '/Users/yaolun/test/'
ConstructInputASURV(fitting_table, line1, line2, outdir)
k, s = RunandParseASURV(code_dir, outdir)
print(k,s)

20 20
b'                           PROBLEMS ARE FINISHED\n'
[]
(2.802, 0.0051) (0.644, 0.005)


In [101]:
import itertools
from astropy.io import ascii
fitting_table = '/Volumes/SD-Mac/CDF_archive_v2_lines.txt'
code_dir = '/Users/yaolun/astro_tools/'
outdir = '/Users/yaolun/astro_tools/'
data = ascii.read(fitting_table)['Line'].data
co_list = list(set(list(filter(lambda k: k.startswith('CO'), data))))

co_list = co_list[-10:]

# print(list(itertools.combinations(co_list, 2)))

co_pair_list = list(itertools.combinations(co_list, 2))
print(len(co_pair_list))

foo = open(outdir+'co_correlation.txt', 'w')
foo.write('{:<10s}{:<10s}{:<10s}{:<10s}{:<10s}{:<10s}\n'.format('Line1','Line2','Ken.z','Ken.prob','Spr.rho','Spr.prob'))

for pair in co_pair_list:
    line1 = pair[0]
    line2 = pair[1]
    
    print(line1, line2)
    
    flag = ConstructInputASURV(fitting_table, line1, line2, outdir)
    if flag:
        k, s = RunandParseASURV(code_dir, outdir)
        print(line1+'/'+line2, k, s)
    
        foo.write('{:<10s}{:<10s}{:<10.3f}{:<10.3f}{:<10.3f}{:<10.3f}\n'.format(line1,line2,k[0],k[1],s[0],s[1]))
    else:
        foo.write('{:<10s}{:<10s}{:<10.3f}{:<10.3f}{:<10.3f}{:<10.3f}\n'.format(line1,line2,0,0,0,0))
    
foo.close()

45
CO16-15 CO31-30
b'                           PROBLEMS ARE FINISHED\n'
[]
CO16-15/CO31-30 (2.303, 0.0213) (0.53, 0.0209)
CO16-15 CO8-7
b'                           PROBLEMS ARE FINISHED\n'
[]
CO16-15/CO8-7 (4.237, 0.0) (0.851, 0.0002)
CO16-15 CO13-12
b'                           PROBLEMS ARE FINISHED\n'
[]
CO16-15/CO13-12 (4.453, 0.0) (0.879, 0.0001)
CO16-15 CO45-44
CO16-15 CO14-13
b'                           PROBLEMS ARE FINISHED\n'
[]
CO16-15/CO14-13 (5.348, 0.0) (0.971, 0.0)
CO16-15 CO47-46
CO16-15 CO5-4
b'                           PROBLEMS ARE FINISHED\n'
[]
CO16-15/CO5-4 (3.803, 0.0001) (0.803, 0.0005)
CO16-15 CO19-18
b'                           PROBLEMS ARE FINISHED\n'
[]
CO16-15/CO19-18 (5.421, 0.0) (0.969, 0.0)
CO16-15 CO32-31
b'                           PROBLEMS ARE FINISHED\n'
[]
CO16-15/CO32-31 (2.795, 0.0052) (0.615, 0.0074)
CO31-30 CO8-7
b'                           PROBLEMS ARE FINISHED\n'
[]
CO31-30/CO8-7 (1.778, 0.0754) (0.41, 0.0742)
CO31-30 CO13-12
b'           

In [ ]:
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from astropy.io import ascii
data = ascii.read(outdir+'co.dat', names=['ufl', line1, line2])

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

ax.plot(np.log10(data[line1][data['ufl'] == 0]), np.log10(data[line2][data['ufl'] == 0]), 'o',   mec='b', mfc='None',
         markeredgewidth=1)
# upper limit on x-axis
ax.errorbar(np.log10(data[line1][data['ufl'] == -2]), np.log10(data[line2][data['ufl'] == -2]),
             xerr=np.array([0.1]*len(data[line1][data['ufl'] == -2])), xuplims=True, fmt='o',   mec='b', mfc='None',
             markeredgewidth=1, color='b', elinewidth=1)
# # upper limit on y-axis
ax.errorbar(np.log10(data[line1][data['ufl'] == -1]), np.log10(data[line2][data['ufl'] == -1]),
             yerr=np.array([0.1]*len(data[line1][data['ufl'] == -1])), uplims=True, fmt='o',   mec='b', mfc='None',
             markeredgewidth=1, color='b', elinewidth=1)
# upper limit on both x-axis and y-axis
ax.errorbar(np.log10(data[line1][data['ufl'] == -3]), np.log10(data[line2][data['ufl'] == -3]),
             xerr=np.array([0.1]*len(data[line1][data['ufl'] == -3])), 
             yerr=np.array([0.1]*len(data[line1][data['ufl'] == -3])),
             xuplims=True, uplims=True, fmt='o', mec='b', mfc='None',
             markeredgewidth=1, color='b', elinewidth=1)
ax.set_xlabel('CO J=24-23 [log(W m$^{-2}$)]', fontsize=18)
ax.set_ylabel('CO J=30-29 [log(W m$^{-2}$)]', fontsize=18)
ax.tick_params('both',labelsize=16,which='major')
